In [1]:
# %pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'tax-index'
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}

    질문: {{question}}

""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [9]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [ ]:

retriever.invoke(query)

[Document(id='53ccfbb3-cd41-45d2-bcef-c009994759c5', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='04461a69-0bc0-44a1-8f82-c3d7a28ef5d7', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='db32e3f9-a0a0-498c-9adb-cb875ae7cf05', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|---------------------------------

In [ ]:



prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:

prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template=""" 
[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해 주세요.  

[Context]
{context}

Question: {question}
"""
)

In [15]:
ai_message = qa_chain.invoke({"query": query})

In [16]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '연봉 5천만원인 직장인의 소득세를 계산하기 위해서 종합소득 과세 표준에 따른 세율을 적용해야 합니다. 우선, 연봉 5천만원이 과세표준이 되는 경우라면, 해당 금액을 기준으로 소득세를 계산합니다.\n\n1. 연봉 5천만원은 1,400만원을 초과하므로, 두 번째 구간에 해당합니다: 1,400만원 초과 5,000만원 이하.\n\n2. 따라서 세금은 다음과 같이 계산됩니다:\n\n   - 기본 세금: 84만원\n   - 추가 세금: (5,000만원 - 1,400만원) × 15% = 3,600만원 × 15% = 540만원\n\n3. 총 소득세 = 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5천만원인 직장인의 소득세는 624만원입니다.'}

In [22]:
new_question = dictionary_chain.invoke({"question": query})
new_question

'질문: 연봉 5천만원 거주자의 소득세는 얼마인가?'

In [19]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인의 소득세는 얼마인가?"

In [20]:
ai_message = tax_chain.invoke({"question": query})

In [21]:
ai_message

{'query': '질문: 연봉 5천만원 거주자의 소득세는 얼마인가?',
 'result': '한국의 소득세는 누진세 구조로 되어 있으며 연간 소득에 따라 세율이 달라집니다. 질문에서 제공된 정보가 제한적이며 모든 소득 구간에 대한 세율이 주어지지 않았습니다. 따라서 일반적인 소득세율을 참고하여 대략적으로 계산해보겠습니다.\n\n2023년 기준 한국의 종합소득세율은 다음과 같습니다:\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억5천만원 이하: 35%\n- 1억5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과 ~ 10억원 이하: 42%\n- 10억원 초과: 45%\n\n연봉 5,000만원의 경우 소득세를 대략적으로 계산해보면:\n\n1. 1,200만원까지: 1,200만원 x 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원까지: (4,600만원 - 1,200만원) x 15% = 510만원\n3. 4,600만원 초과 ~ 5,000만원까지: (5,000만원 - 4,600만원) x 24% = 96만원\n\n따라서 총 소득세는 72만원 + 510만원 + 96만원 = 678만원이 됩니다.\n\n다만, 실제 소득세는 다양한 공제와 세액공제가 적용될 수 있으므로, 정확한 금액은 변동할 수 있습니다. 추가적인 공제 사항이나 개별 상황에 따른 소득세 계산을 위해 세무 전문가에게 상담받는 것이 좋습니다.'}